<a href="https://colab.research.google.com/github/Pholoxo/InventoryProject/blob/master/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Description: Build a movie recommendation engine using python

In [ ]:
#Import the libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
#Load the data
from google.colab import files
uploaded = files.upload()

Saving imdb_top_1000.csv to imdb_top_1000.csv


In [20]:
#Store the data
df = pd.read_csv('imdb_top_1000.csv')
#Show the first 3 rows of data
df.head(3)

,Movie_id,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,Actors,No_of_Votes,Gross
0,0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2343110,"28,341,469"
1,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1620367,"134,966,411"
2,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2303232,"534,858,444"


In [21]:
#Get a count of the number of rows/movies in the data set and the number of columns
df.shape

(1000, 17)

In [22]:
#Create a list of important columns for the recommendation engine
columns = ['Actors', 'Director', 'Genre', 'Series_Title']

In [23]:
df[columns].head(3)

,Actors,Director,Genre,Series_Title
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Frank Darabont,Drama,The Shawshank Redemption
1,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",Francis Ford Coppola,"Crime, Drama",The Godfather
2,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Christopher Nolan,"Action, Crime, Drama",The Dark Knight


In [25]:
#Check for any missing values in the important columns
df[columns].isnull().values.any()

False

In [26]:
#Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(data['Actors'][i] + ' ' + data['Director'][i] + ' ' + data['Genre'][i] + ' ' + data['Series_Title'][i])

  return important_features

In [27]:
#Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#Show the data
df.head(3)

,Movie_id,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,Actors,No_of_Votes,Gross,important_features
0,0,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",2343110,"28,341,469","Tim Robbins, Morgan Freeman, Bob Gunton, Willi..."
1,1,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,"Marlon Brando, Al Pacino, James Caan, Diane Ke...",1620367,"134,966,411","Marlon Brando, Al Pacino, James Caan, Diane Ke..."
2,2,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",2303232,"534,858,444","Christian Bale, Heath Ledger, Aaron Eckhart, M..."


In [28]:
#Convert the text to a matrix of token counts
cm = CountVectorizer().fit_transform(df['important_features'])

In [30]:
#Get the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)

[[1.         0.13801311 0.13363062 ... 0.06482037 0.14824986 0.06681531]
 [0.13801311 1.         0.19364917 ... 0.06262243 0.07161149 0.12909944]
 [0.13363062 0.19364917 1.         ... 0.06063391 0.06933752 0.125     ]
 ...
 [0.06482037 0.06262243 0.06063391 ... 1.         0.13453456 0.        ]
 [0.14824986 0.07161149 0.06933752 ... 0.13453456 1.         0.13867505]
 [0.06681531 0.12909944 0.125      ... 0.         0.13867505 1.        ]]


In [32]:
#Get the shape of the cosine similarity matrix
cs.shape

(1000, 1000)

In [35]:
#Get the title of the movie that the user likes
title = 'Pulp Fiction'

#Find the movies id
movie_id = df[df.Series_Title == title]['Movie_id'].values[0]

In [43]:
#Create a list of enumerations for the similarity score
scores = list(enumerate(cs[movie_id]))

In [45]:
#Sort the list
sorted_scores = sorted(scores, key = lambda x:x[1], reverse = True)
sorted_scores = sorted_scores[1:]

In [46]:
#Print the sorted scores
print(sorted_scores)

[(241, 0.40089186286863654), (584, 0.3889222341312986), (369, 0.33407655239053047), (963, 0.3241018617760822), (103, 0.2760262237369417), (850, 0.2672612419124244), (713, 0.23904572186687872), (805, 0.21821789023599242), (62, 0.2142857142857143), (171, 0.2142857142857143), (364, 0.2142857142857143), (450, 0.2142857142857143), (849, 0.2142857142857143), (4, 0.20701966780270625), (93, 0.20701966780270625), (180, 0.20701966780270625), (598, 0.20701966780270625), (654, 0.20701966780270625), (699, 0.20701966780270625), (710, 0.20701966780270625), (811, 0.20701966780270625), (416, 0.2004459314343183), (680, 0.2004459314343183), (248, 0.19446111706564934), (276, 0.19446111706564934), (284, 0.19446111706564934), (523, 0.19446111706564934), (553, 0.19446111706564934), (827, 0.19446111706564934), (879, 0.19446111706564934), (978, 0.19446111706564934), (980, 0.19446111706564934), (986, 0.19446111706564934), (278, 0.18898223650461363), (457, 0.18898223650461363), (669, 0.18898223650461363), (674, 

In [47]:
#Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended movies to', title, 'are:\n')
for item in sorted_scores:
  movie_title = df[df.Movie_id == item[0]]['Series_Title'].values[0]
  print(j+1, movie_title)
  j = j + 1
  if j > 6:
    break

The 7 most recommended movies to Pulp Fiction are:

1 Kill Bill: Vol. 1
2 The Hateful Eight
3 Kill Bill: Vol. 2
4 Die Hard: With a Vengeance
5 Reservoir Dogs
6 Enter the Dragon
7 Stagecoach
